# Cáculo excato para diferente número de sitios, P.

In [ ]:
%matplotlib inline
import numpy as np
from qutip import *
import matplotlib.pyplot as plt
from sympy.utilities.iterables import multiset_permutations
from time import time

In [ ]:
def Is(i): return [qeye(2) for j in range(0, i)]
def b(i): return [basis(2,0) for j in range(0, i)]
def sigz(i): return[sigmaz() for j in range(0,i)]
def d(N,i): return tensor(sigz(i)+[destroy(2)]+ Is(N-i-1))

In [ ]:
def factorial(n):
    productoria = 1
    for i in range(2, n+1):
        productoria *= i
    return productoria

In [ ]:
def combinatorio(n, k):
    return factorial(n)//(factorial(n-k)*factorial(k)) 

In [ ]:
def base_comp(Sites, N_electrones):
    q=b(2*Sites)
    for i in range(0,N_electrones):
        q[i]=create(2)*q[i]
    return list(multiset_permutations(q))

In [ ]:
def n_operator(sites, n_electrones):
    n=[[0 for x in range(2)] for y in range (sites)]
    l=0
    for i in range(sites):
        for j in range(2):
            n[i][j]=d(2*sites,l).dag()*d(2*sites,l)
            l+=1
    
    return n

In [ ]:
def spin_total(base1):
    s=0
    for i in range (len(base1)):
        if(base1[i]==basis(2,1) and i%2==0):
            s+=-0.5
        elif (base1[i]==basis(2,1) and i%2!=0):
            s+=0.5         
    return s

In [ ]:
def kinetic(t, sites):
    
    T=0
    
    for i in range (0,2*sites):
        for j in range (0,2*sites):
            T+=t[i][j]*d(2*sites,i).dag()*d(2*sites,j)
            
    return T

In [ ]:
def local_potential(v,sites):
    V=0
    l=0
    for i in range(sites):
        for j in range (2):
            V+=v[i]*d(2*sites,l).dag()*d(2*sites,l)
            l+=1
    return V  

In [ ]:
def two_body(U, sites):
    S=0
    
    for i in range(sites):
        S+=U*d(2*sites, 2*i).dag()*d(2*sites, 2*i)*d(2*sites, 2*i+1).dag()*d(2*sites, 2*i+1)
    return S

In [ ]:
N_e=2
P=[2, 4, 6, 8, 10]            
U=0.2

In [ ]:
E=[]
tiempo=[]

for j in range (0, len(P)):
    
    t_0=time()
    
    A=0
    base_total=base_comp(P[j],N_e)

    for i in range (combinatorio(2*P[j],N_e)):
        A+=tensor(base_total[i])*basis(combinatorio(2*P[j],N_e),i).dag()
    
    t = [[0 for x in range(2*P[j])] for y in range(2*P[j])]
    v=np.zeros(P[j])
    
    for i in range (0, 2*P[j]-2):
        t[i][i+2]=t[i+2][i]=-0.5
    
    h_1=A.dag()*kinetic(t,P[j])*A
    h_2=A.dag()*two_body(U,P[j])*A
    
    
    for i in range (0,P[j]):
        v[i]=1.
            
    h_3=A.dag()*local_potential(v,P[j])*A
    H=h_1+h_2+h_3
    evals, ekets= H.eigenstates()
        
    t_1=time()-t_0
    
    E.append(evals[0])
    tiempo.append(t_1)

np.savetxt("E_t_P_exact.txt", np.c_[E,tiempo,P])